# Simulations with Gammapy

Here we show how to simulate the properties of one or more sources in a given field of view with the use of Gammapy. The basic ingredientes are:
- a skymodel to simulate, i.e. the source properties (spectral, spatial and temporal) plus the background;
- a dataset, which carries information on the FoV geometry, the energy range, the IRF properties and so on;
- an observation object, which defines the exposure time, the starting time of the simulation, the pointing direction, the array configuration etc;

Then we can set the simulator class and perform the event sampling on the given sky-model.

Let's start with some basic imports:

In [ ]:
import matplotlib.pyplot as plt

import time
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from gammapy.data import Observation, observatory_locations, FixedPointingInfo
from gammapy.datasets import MapDataset, MapDatasetEventSampler
from gammapy.irf import load_irf_dict_from_file
from gammapy.makers import MapDatasetMaker
from gammapy.maps import MapAxis, RegionNDMap, WcsGeom, Map
from gammapy.modeling.models import (
    ConstantSpectralModel,
    FoVBackgroundModel,
    LightCurveTemplateTemporalModel,
    GaussianTemporalModel,
    ConstantTemporalModel,
    ExpDecayTemporalModel,
    PointSpatialModel,
    GaussianSpatialModel,
    PowerLawSpectralModel,
    SkyModel,
    Models
)

### Creation of the sky-model

First of all, we need define the source model. For simplicity, we consider a point-like source with a powerlaw spectral shape, at a random position (here Ra, Dec = 335°, 3°). We will not include temporal variability (but if you like, there are some examples in the box below..). For more models, please visit the Gammapy website at https://docs.gammapy.org/1.2/user-guide/model-gallery/index.html .

In [ ]:
target = SkyCoord("335 deg", "3 deg", unit="deg", frame="icrs")
spatial_model = PointSpatialModel.from_position(target)

spectral_model = PowerLawSpectralModel(index=2, amplitude="1e-11 cm-2 s-1 TeV-1", reference="1 TeV")
temporal_model = ConstantTemporalModel(const=1)

Gammapy collects the spatial, temporal and spectral information into a single object, called `SkyModel`:

In [ ]:
source_model = SkyModel(spectral_model=spectral_model,
                        spatial_model=spatial_model,
                        temporal_model=temporal_model,
                        name="fake_src")
print(source_model)

We need to include the background as well. Let's consider the model taken from the IRF:

In [ ]:
bkg = FoVBackgroundModel(dataset_name="my-dataset")

The full model (i.e. source plus background) can be stored into the `Models` object. It can handle a number of models!

In [ ]:
full_model = Models([source_model, bkg])
print(full_model)

## Create the dataset

Let's define the energy range, the pointing coordinates, the exposure time and the size of the field of view. Then we can load the IRF to be used for the simulation and we will create the observation properties:

In [ ]:
# reconstructed energy:
energy_axis = MapAxis.from_energy_bounds("0.05 TeV", "150 TeV",
                                         nbin=5, per_decade=True)

# we pass also an axis to calculate the energy dispersion:
energy_axis_true = MapAxis.from_energy_bounds("0.01 TeV", "200 TeV", 
                                         nbin=10, per_decade=True, 
                                         name="energy_true")
migra_axis = MapAxis.from_bounds(0.5, 2, nbin=150, node_type="edges", name="migra")

In [ ]:
# Let's assume an exposure time of 5 hours:
livetime = 5 * u.hr

In [ ]:
# Let's load an IRF for the South array:
path = "$GAMMAPY_DATA/cta-caldb/Prod5-South-20deg-AverageAz-14MSTs37SSTs.180000s-v0.1.fits.gz"
irf = load_irf_dict_from_file(path)

In [ ]:
# We can define a FoV geometry of a square of 5° by side and a square pixel of 0.01° by side, centered 
# at the pointing position:

geom = WcsGeom.create(
            skydir=target,
            width=(5, 5),
            binsz=0.01, 
            frame="galactic",
            axes=[energy_axis],
            )

Now we can create the `Observation` object to store all the needed information for the observation. We assume that the observation starts today: 

In [ ]:
time_ref = Time("2024-06-25T00:00:00", format="isot", scale="utc")
pointing = FixedPointingInfo(
    fixed_icrs=target.icrs,
)

observation = Observation.create(
            obs_id="0001",
            pointing=pointing,
            livetime=livetime,
            irfs=irf,
            reference_time=time_ref,
            )

Finally, we can create the `Dataset` object and we fill it with the full sky-model:

In [ ]:
empty = MapDataset.create(
            geom,
            energy_axis_true=energy_axis_true,
            migra_axis=migra_axis,
            name="my-dataset",
            )
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
dataset = maker.run(empty, observation)

In [ ]:
dataset.models = full_model

In [ ]:
print(dataset)

## Let's simulate the events:

We call the `MapDatasetEventSampler` class, the core of the simulations in Gammapy, and we set a seed for the random generator. Then, we sampled the source and background events

In [ ]:
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

In [ ]:
events.table

In [ ]:
# the number of source events is:
len(events.table[events.table["MC_ID"] == 1])

In [ ]:
# the number of background events is:
len(events.table[events.table["MC_ID"] == 0])

Events can be inspected with a quick look using the function `peek`:

In [ ]:
events.peek()

Now that we have generated the sampled events, we can proceed to fit them for example. However, before doing that, we need to fill the `dataset` with them:

In [ ]:
counts = Map.from_geom(geom)
counts.fill_events(events)
dataset.counts = counts

In [ ]:
dataset.fake()

In [ ]:
print(dataset)

## Open problems: 

6a) Simulate a source with an extended morphology (for example, a Gaussian spatial extensions with size 0.5°) and a powerlaw spectral shape ($\Gamma=2$, amplitude=$10^{-13}$ cm$^{-2}$ s$^{-1}$ TeV$^{-1}$ at 1 TeV). Determine then minimum exposure time needed to constrain the extension of the source to better than 10%; for simplicity, consider a FoV dimension of $2.5°\times2.5°$ and fix the source coordinates when fitting the model to the sampled data.
 
6b) Consider the extended Galactic source RX J1713-3946, located at 1 kpc from us. Its coordinates are l, b = 347.313895°, -0.459872° and its spectrum can be modeled with a powerlaw of $\Gamma=2.06$ and amplitude of $2.3\times10^{-11}$ cm$^{-2}$ s$^{-1}$ TeV$^{-1}$ at 1 TeV.
Simulate it and fit the sampled events to estimate the constraints on the size of its morpholgy. Then, move the source to larger distances, rescaling the powerlaw amplitude and the source size accordingly, and estimate its morphology. Calculate how the constraints on the size change with the distance. For simplicity, consider an exposure time of 10 hours and fix the source coordinates when fitting the model.